In [1]:
f = open('WhatsApp Chat with Dark Future 💥.txt', 'r', encoding='utf-8')
data = f.read()
data

'02/10/24, 7:09\u202fpm - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.\n02/10/24, 6:40\u202fpm - Abu Naiim BA created group "Dark Future 💥"\n02/10/24, 7:09\u202fpm - You joined using this group\'s invite link\n02/10/24, 7:13\u202fpm - Sourav BA added Lubna BA\n02/10/24, 7:25\u202fpm - Tanjila BA joined using this group\'s invite link\n03/10/24, 10:38\u202fam - Shoaib Hoque: https://dev.mrdbourke.com/tensorflow-deep-learning/\n03/10/24, 8:46\u202fpm - Shoaib Hoque: Accha sunday to sir ashbena..  Oidin ki apnara ashben naki keo.. Karon ek dui ashle, bakider na dekhle majammel vhai call diye chillachilli korbe.. Ar keo na gele shobai ekshathe bolte parbo sir bolse oidin class hobena tai amra keo ashinai.\n\nTai keo ashle janayen.. Taile ashbo arki nahoy ekdin rest e thaka jabe\n03/10/24, 8:49\u202fpm - Mynul BA: vai sabai jodi jete na chay tahole amio jabo na\n03/10/24, 8:51\u202fpm - Saimu BA: 

In [2]:
dummy = data.split('\n')
dummy

['02/10/24, 7:09\u202fpm - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.',
 '02/10/24, 6:40\u202fpm - Abu Naiim BA created group "Dark Future 💥"',
 "02/10/24, 7:09\u202fpm - You joined using this group's invite link",
 '02/10/24, 7:13\u202fpm - Sourav BA added Lubna BA',
 "02/10/24, 7:25\u202fpm - Tanjila BA joined using this group's invite link",
 '03/10/24, 10:38\u202fam - Shoaib Hoque: https://dev.mrdbourke.com/tensorflow-deep-learning/',
 '03/10/24, 8:46\u202fpm - Shoaib Hoque: Accha sunday to sir ashbena..  Oidin ki apnara ashben naki keo.. Karon ek dui ashle, bakider na dekhle majammel vhai call diye chillachilli korbe.. Ar keo na gele shobai ekshathe bolte parbo sir bolse oidin class hobena tai amra keo ashinai.',
 '',
 'Tai keo ashle janayen.. Taile ashbo arki nahoy ekdin rest e thaka jabe',
 '03/10/24, 8:49\u202fpm - Mynul BA: vai sabai jodi jete na chay tahole amio jabo na',
 '03/10/

In [5]:
import re 
pattern = r'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'
messages = re.split(pattern,data)[1:]
print(len(messages))

## extracting dates as well in different col

dates = re.findall(pattern,data)
print(len(dates))

0
0


In [2]:
import streamlit as st
import numpy as np
import seaborn as sns
import pandas as pd
import re

def gettimedate(string):
    string = string.split(',')
    date, time = string[0], string[1]
    time = time.split('-')[0].strip()
    return date + " " + time

def getstring(text):
    return text.split('\n')[0]

def preprocess(data):
    # Updated pattern to include `\u202f`
    pattern = r'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\u202f?[ap]m\s-\s'
    
    messages = re.split(pattern, data)[1:]
    dates = re.findall(pattern, data)

    # Create DataFrame
    df = pd.DataFrame({'user_messages': messages, 'message_date': dates})
    
    # Apply `gettimedate`
    df['message_date'] = df['message_date'].apply(gettimedate)
    df.rename(columns={'message_date': 'date'}, inplace=True)

    users = []
    messages = []
    for message in df['user_messages']:
        entry = re.split(r'([\w\W]+?):\s', message)
        if entry[1:]:
            users.append(entry[1])
            messages.append(entry[2])
        else:
            users.append("Group Notification")
            messages.append(entry[0])

    df['user'] = users
    df['message'] = messages
    df['messages'] = df['message'].apply(getstring)
    
    # Drop and reorder columns
    df = df.drop(['user_messages'], axis=1)
    df = df[['message', 'date', 'user']]

    # Renaming and adding date columns
    df = df.rename(columns={'message': 'Message', 'date': 'Date'})
    df['Only_Date'] = pd.to_datetime(df['Date']).dt.date
    df['Year'] = pd.to_datetime(df['Date']).dt.year
    df['Month_Num'] = pd.to_datetime(df['Date']).dt.month
    df['Month'] = pd.to_datetime(df['Date']).dt.month_name()
    df['Day'] = pd.to_datetime(df['Date']).dt.day
    df['Day_Name'] = pd.to_datetime(df['Date']).dt.day_name()
    df['Hour'] = pd.to_datetime(df['Date']).dt.hour
    df['Minute'] = pd.to_datetime(df['Date']).dt.minute

    return df

In [ ]:
from urlextract import URLExtract
import pandas as pd
from collections import Counter
from wordcloud import WordCloud
import emoji

extract = URLExtract()

def fetchstats(selected_user, df):

    # if the selected user is a specific user, then make changes in dataframe else do not change
    if selected_user != 'Overall':
        df = df[df['user'] == selected_user]

    num_messages = df.shape[0]
    words = []

    for message in df['Message']:
        words.extend(message.split())

    # Counting the number of media files shared
    mediaommitted = df[df['Message'] == '<Media omitted>']

    # Number of links shared

    links = []
    for message in df['Message']:
        links.extend(extract.find_urls(message))
    
    return num_messages, len(words), mediaommitted.shape[0], len(links)


In [3]:
df = preprocess(data)
df

C:\Users\Shoaib\AppData\Local\Temp\ipykernel_12680\1872724500.py:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Only_Date'] = pd.to_datetime(df['Date']).dt.date
C:\Users\Shoaib\AppData\Local\Temp\ipykernel_12680\1872724500.py:52: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Year'] = pd.to_datetime(df['Date']).dt.year
C:\Users\Shoaib\AppData\Local\Temp\ipykernel_12680\1872724500.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Month_Num'] = pd.to_datetime(df['Date']).dt.month
C:\Users\Shoaib\AppData\Local\Temp\ipykernel_12680\1872724500.py:54: UserWarning

,Message,Date,user,Only_Date,Year,Month_Num,Month,Day,Day_Name,Hour,Minute
0,Messages and calls are end-to-end encrypted. N...,02/10/24 7:09 pm,Group Notification,2024-02-10,2024,2,February,10,Saturday,19,9
1,"Abu Naiim BA created group ""Dark Future 💥""\n",02/10/24 6:40 pm,Group Notification,2024-02-10,2024,2,February,10,Saturday,18,40
2,You joined using this group's invite link\n,02/10/24 7:09 pm,Group Notification,2024-02-10,2024,2,February,10,Saturday,19,9
3,Sourav BA added Lubna BA\n,02/10/24 7:13 pm,Group Notification,2024-02-10,2024,2,February,10,Saturday,19,13
4,Tanjila BA joined using this group's invite li...,02/10/24 7:25 pm,Group Notification,2024-02-10,2024,2,February,10,Saturday,19,25
...,...,...,...,...,...,...,...,...,...,...,...
524,welcome bro\n,30/10/24 2:50 pm,+880 1540-523032,2024-10-30,2024,10,October,30,Wednesday,14,50
525,treat dio\n,30/10/24 2:50 pm,+880 1540-523032,2024-10-30,2024,10,October,30,Wednesday,14,50
526,<Media omitted>\n,30/10/24 2:51 pm,Abu Naiim BA,2024-10-30,2024,10,October,30,Wednesday,14,51
527,Dekha jaak\n,30/10/24 2:59 pm,Sazzad BA,2024-10-30,2024,10,October,30,Wednesday,14,59
